# Prep the datasets to use in analysis

1. Clean both HMDA and census datasets.
1. Reduce the HMDA loan-level file to a bank-tract dataset.
1. Merge the bank-tract dataset with the census data.


In [1]:
import os
import csv
import pandas as pd
import numpy as np

os.makedirs('../input_data_clean/',exist_ok=True)


## HMDA Cleaning

- Clean the var types
- Better variable values (replace loan type 1 with name)

In [57]:
#todo when complete, delete this comment and uncomment the if, indent everything below
if not os.path.exists('../input_data_clean/state_AZ-CA_clean.csv'):

    hmda = pd.read_csv('../input_data/state_AZ-CA.csv')

    ##################################################################
    # SAMPLE RESTRICTION: ACTION TAKEN = 1,2,3 ONLY
    ##################################################################

    hmda = hmda.query('action_taken <= 3') 

    ##################################################################
    # START CLEANING
    ##################################################################

    hmda = hmda.replace(["Exempt","1111",1111,"99999",99999,"8888",8888,"9999",9999,"999.99",999.99,"NA","nan"], np.nan)  

    hmda['simplified_applicant_age_above_62'] = hmda['applicant_age_above_62'].map({'Yes': 1, 'No': 0})
    hmda.drop(columns = ['applicant_age_above_62'],inplace=True)

    hmda['state_simplified'] = hmda['state_code'].map({'AZ': 1, 'CA': 0})


    hmda["action_taken"] = hmda["action_taken"].replace({
        1:"Loan Originated",
        2:"Application approved but not accepted",
        3:"Application denied",
        4:"Application withdrawn by applicant",
        5:"File closed for incompleteness",
        6:"Purchased loan",
        7:"Preapproval request denied",
        8:"Preapproval request approved but not accepted"
    })
    hmda["purchaser_type"] = hmda["purchaser_type"].replace({
        0:"Not applicable",
        1:"Fannie Mae",
        2:"Ginnie Mae",
        3:"Freddie Mac",
        4:"Farmer Mac",
        5:"Private securitizer",
        6:"Commercial bank savings bank or savings association",
        71:"Credit union mortgage company or finance company",
        72:"Life insurance company",
        8:"Affiliate institution",
        9:"Other Type of purchaser"
    })
    hmda["preapproval"] = hmda["preapproval"].replace({
        1:"Preapproval requested",
        2:"Preapproval not requested"
    })
    hmda["loant_type"] = hmda["loan_type"].replace({
        1:"Conventional",
        2:"Federal Housing Administration FHA",
        3:"Veterans Affiars guaranteed VA",
        4:"USDA Rural Housing Service or the Farm Service Agency guaranteed RHS or FSA"
    })
    hmda["loan_purpose"] = hmda["loan_purpose"].replace({
        1:"Home purchase",
        2:"Home improvement",
        31:"Refinancing",
        32:"Cash-out refinancing",
        4:"Other purpose",
        5:"Not applicable",
    })
    hmda["lien_status"] = hmda["lien_status"].replace({
        1:"Secured by a first lien",
        2:"Secured by a subordinate lien"
    })
    hmda["hoepa_status"] = hmda["hoepa_status"].replace({
        1:"High-cost mortgage",
        2:"Not a high-cost mortgage",
        3:"Not applicable"
    })
    hmda["reverse_mortgage"] = hmda["reverse_mortgage"].replace({
        1:"Reverse mortgage",
        2:"Not a reverse mortgage",
        1111:"Exempt"
    })
    hmda["open-end_line_of_credit"] = hmda["open-end_line_of_credit"].replace({
        1:"Open-end line of credit",
        2:"Not an open-end line of credit",
        1111:"Exempt"
    })
    hmda["business_or_commercial_purpose"] = hmda["business_or_commercial_purpose"].replace({
        1:"Primarily for a business or commercial purpose",
        2:"Not primarily for a business or commercial purpose",
        1111:"Exempt"
    })
    hmda["negative_amortization"] = hmda["negative_amortization"].replace({
        1:"Negative amortization",
        2:"No negative amortization",
        1111:"Exempt"
    })
    hmda["interest_only_payment"] = hmda["interest_only_payment"].replace({
        1:"Interest-only payment",
        2:"No interest-only payments",
        1111:"Exempt"
    })
    hmda["balloon_payment"] = hmda["balloon_payment"].replace({
        1:"Balloon payment",
        2:"No balloon payment",
        1111:"Exempt"
    })
    hmda["other_nonamortizing_features"] = hmda["other_nonamortizing_features"].replace({
        1:"Other non-fully amortizing features",
        2:"No other non-fully amortizing features",
        1111:"Exempt"
    })
    hmda["construction_method"] = hmda["construction_method"].replace({
        1:"Site-built",
        2:"Manufactured Home",
        1111:"Exempt"
    })
    hmda["occupancy_type"] = hmda["occupancy_type"].replace({
        1:"Principal residence",
        2:"Second residence",
        3:"Investment Property"
    })
    hmda["applicant_credit_score_type"] = hmda["applicant_credit_score_type"].replace({
        1:"Equifax Beacon 5.0",
        2:"Experian Fair Isaac",
        3:"FICO Risk Score Classic 04",
        4:"FICO Risk Score Classic 98",
        5:"VintageScore 2.0",
        6:"VintageScore 3.0",
        7:"More than one credit scoring model",
        8:"Other credit scoring model",
        9:"Not Applicable"
    })
    hmda["co-applicant_credit_score_type"] = hmda["co-applicant_credit_score_type"].replace({
        1:"Equifax Beacon 5.0",
        2:"Experian Fair Isaac",
        3:"FICO Risk Score Classic 04",
        4:"FICO Risk Score Classic 98",
        5:"VintageScore 2.0",
        6:"VintageScore 3.0",
        7:"More than one credit scoring model",
        8:"Other credit scoring model",
        9:"Not Applicable",
        10:"No co-applicant"
    })
    #did not include eth-4 or 5
    eths_to_modify = ["applicant_ethnicity-1","applicant_ethnicity-2","applicant_ethnicity-3",
                "co-applicant_ethnicity-1","co-applicant_ethnicity-2","co-applicant_ethnicity-3"]
    eths_replace={
        1:"Hispanic or Latino",
        2:"Mexican",
        3:"Puerto Rican",
        4:"Cuban",
        5:"Other Hispanic or Latino",
        2:"Not Hispanic or Latino",
        3:"Info not provided by applicant in mail internet or telephone appplication",
        4:"Not Applicable"
    }
    for col in eths_to_modify:
        hmda[col] = hmda[col].replace(eths_replace)


    race_to_modify = ["applicant_race-1","applicant_race-2","applicant_race-3",
                      "applicant_race-4","applicant_race-5",
                      "co-applicant_race-1","co-applicant_race-2","co-applicant_race-3",
                      "co-applicant_race-4","co-applicant_race-5"]
    race_replace={
        1:"American Inidian or Alaska Native",
        2:"Asian",
        21:"Asian Indian",
        22:"Chinese",
        23:"Filipino",
        24:"Japanese",
        25:"Korean",
        26:"Vietnamese",
        27:"Other Asian",
        3:"Black or African American",
        4:"Native Hawaiian or Other Pacific Islander",
        41:"Native Hawaiian",
        42:"Guamanian",
        43:"Samoan",
        44:"Other Pacific Islander",
        5:"White",
        6:"Info not provided by applicant in mail internet or telephone appplication",
        7:"Not Applicable",
        8:"No co-applicant"
    }
    for col in race_to_modify:
        hmda[col] = hmda[col].replace(race_replace)

    obs_to_modify = ["applicant_ethnicity_observed","co-applicant_ethnicity_observed","applicant_race_observed",
                "co-applicant_race_observed","applicant_sex_observed","co-applicant_sex_observed"]
    obs_replace={
        1:"Collected on the basis of visual observation or surname",
        2:"Not collected on the basis of visual observation or surname",
        3:"Not applicable",
        4:"No co-applicant"
    }
    for col in obs_to_modify:
        hmda[col] = hmda[col].replace(obs_replace)

    hmda["applicant_sex"] = hmda["applicant_sex"].replace({
        1:"Male",
        2:"Female",
        3:"Information not provided by applicant in mail internet or telephone application",
        4:"Not applicable",
        5:"No co-applicant",
        6:"Applicant selected both male and female"
    })
    hmda["co-applicant_sex"] = hmda["co-applicant_sex"].replace({
        1:"Male",
        2:"Female",
        3:"Information not provided by applicant in mail internet or telephone application",
        4:"Not applicable",
        5:"No co-applicant",
        6:"Applicant selected both male and female"
    })
    dens_to_modify = ["denial_reason-1","denial_reason-2","denial_reason-3"]
    dens_replace={
        1:"Debt-to-income ratio",
        2:"Employment history",
        3:"Credit history",
        4:"Collateral",
        5:"Insufficient cash downpayment closing costs",
        6:"Unverifiable information",
        7:"Credit application incomplete",
        8:"Mortgage insurance denied",
        9:"Other",
        10:"Not applicable",
        1111:"Exempt"
    }
    for col in dens_to_modify:
        hmda[col] = hmda[col].replace(dens_replace)

    hmda['income'] = hmda['income'].where(hmda['income'] >= 0, np.nan)

    tumapping = {'1': 1, '2': 2, '3': 3, '4': 4,'5-24': 5.24, '25-49': 25.49, '50-99': 50.99, '100-149': 100.149, '>149': 149.9}
    hmda["total_units"] = hmda["total_units"].replace(tumapping)# create a dictionary that maps the binned values to numerical values


    debt_to_incomeratio_mapping = {'<20%': 0.1, '20%-<30%': 0.25, '30%-<36%': 0.32,"36":0.36, 
                                   '37' :0.37, '38': 0.38, '39':0.39, '40':0.4, '41':0.41,
                                   '42' :0.42, '43':0.43, '44':0.44, '45':0.45, '46':0.46,
                                   '47':0.47, '48':0.48, '49':0.49, '50%-60%': 0.55, '>60%':0.65}

    hmda["debt_to_income_ratio2"] = hmda["debt_to_income_ratio"].map(debt_to_incomeratio_mapping)

    hmda[["debt_to_income_ratio2","debt_to_income_ratio"]].count()

    aamapping = {'<25': 25, '25-34': 30, '35-44': 40, '45-54': 50,
                 '55-64': 60, '65-74': 70, '>74': 78,}
    hmda["applicant_age"] = hmda["applicant_age"].replace(aamapping)
    hmda["co-applicant_age"] = hmda["co-applicant_age"].replace(aamapping)
    hmda["co-applicant_age"].describe()

    hmda = hmda.astype({
        'loan_to_value_ratio': float,
        'rate_spread': float,
        'interest_rate': float,
        'prepayment_penalty_term': float,
        'intro_rate_period': float,
        'property_value': float,
        'total_loan_costs': float,
        'total_points_and_fees': float,
        'origination_charges': float,
        'discount_points': float,
        'lender_credits': float,
        'loan_term': float,
        'prepayment_penalty_term': float
    })

    #todo use the output report to continue cleaning
    #todo when done, save it:

    hmda.to_csv('../input_data_clean/state_AZ-CA_clean.csv',index=False)

    # hmda.to_csv('../input_data_clean/state_AZ-CA_clean.csv',index=False)
    # hmda.sample(50000).to_csv('../input_data_clean/state_AZ-CA_clean_MINI.csv',index=False)

    #todo check that it works: load the mini file (read_csv) you just saved and look at it... ok? same index? no extra var?

Save a report on the dataset. This will be useful to refer back to later on.

In [ ]:
# to rerun this, delete the file
if not os.path.exists("mini_HMDA_partial_clean.html"):
    #!pip install -U ydata-profiling 
    from ydata_profiling import ProfileReport
    profile = ProfileReport(hmda, title="HMDA after existing cleaning steps", minimal=True)
    profile.to_file("mini_HMDA_partial_clean.html")
    
# todo after more cleaning... output to outputs/HMDA_profile_report.html instead    

## Census cleaning

1. Rename the variables into something usable.
1. Prepare a `census_tract` variable we can merge with the HMDA data. 

In [44]:
if not os.path.exists('../input_data_clean/census_clean.csv'):

    # this csv: col1 is census var ID name, col2 is the name we are using
    # we are importing this into a dictionary bc pandas rename() wants that structure
    rename_dict ={}
    with open('../input_data/census_vars.csv') as f:
        for row in csv.reader(f):
            rename_dict[row[0]] = row[1]    
            
    census = (
            pd.read_csv('../input_data/census.csv')
            .rename(columns = rename_dict)
    )

    def census_concat(row):
        state  = str(row['state']) .zfill(2)
        county = str(row['county']) .zfill(3)
        tract  = str(row['tract']) .zfill(6)
        return int(state+county+tract)

    census['census_tract'] = census.apply(census_concat, axis=1)

    census.to_csv('../input_data_clean/census_clean.csv',index=False)

## Reduce the loan-level HMDA dataset to a Bank-Tract level dataset

First, create variables we need before the aggregation step. 

In [ ]:
#todo if not os.path.exists('../input_data_clean/bank_tract.csv'):

#todo when all good, redo this with FULL
hmda = pd.read_csv('../input_data_clean/state_AZ-CA_clean.csv')

hmda['approved'] = (hmda['action_taken'] == "Loan Originated") | (hmda['action_taken'] == "Application approved but not accepted") 

hmda['application'] = (hmda['action_taken'] == "Loan Originated") | (hmda['action_taken'] == "Application approved but not accepted") | (hmda['action_taken'] == "Application denied")  

hmda['denial'] = hmda['action_taken'] == "Application denied"

# filter to banks that do 50-200% of BOW's origination volume

counts            = hmda.groupby('lei')['action_taken'].value_counts()
originated_counts = counts.loc[(slice(None), 'Loan Originated')]
lei1_count_range  = (0.5 * originated_counts['QOT5WN9RBKQTFRVKEV31'], 2 * originated_counts['QOT5WN9RBKQTFRVKEV31'])
filtered_counts   = originated_counts[(lei1_count_range[0] <= originated_counts) & (originated_counts <= lei1_count_range[1])]
competitor_leis   = filtered_counts.index.tolist()
hmda = hmda.query('lei in @competitor_leis')

# change to BOW, BMO, Other

def lei_map(lei):
    if lei == 'QOT5WN9RBKQTFRVKEV31':
        return 'Bank of West'
    elif lei == '3Y4U8VZURTYWI1W2K376':
        return 'BMO Harris'
    elif lei == np.nan:
        return np.nan
    else: 
        return 'All Other Banks'

hmda['which_bank'] = hmda['lei'].map(lei_map)

# now  aggregate by area:

bank_tract = hmda.groupby(['which_bank','census_tract']).agg(
    {'approved': 'sum', 
     'application': 'sum', 
     'loan_amount': 'sum',
     'income': 'median',
    }
)

bank_tract = bank_tract.rename(columns={
    'approved': 'num_approved_loans',
    'application': 'num_applications',
    'loan_amount': 'sum_all_loan_amount',
    'income': 'median_all_income',
})

bank_tract.eval('denial_rate = num_approved_loans/num_applications',inplace=True)


bank_tract_approved_loanstats =hmda.query('approved').groupby(['which_bank','census_tract']).agg(
    {'loan_amount': 'sum', 
     'loan_to_value_ratio': 'mean',
     'rate_spread': 'mean',
     'interest_rate': 'mean',
    }
)

bank_tract_approved_loanstats = bank_tract_approved_loanstats.rename(columns={
    'loan_amount': 'sum_approved_loans',
    'loan_to_value_ratio': 'mean_LTV',
    'rate_spread': 'mean_approved_rate_spread',
    'interest_rate': 'mean_approved_interest_rate',
})

bank_tract = bank_tract_approved_loanstats.merge(bank_tract,left_index=True,right_index=True,validate='1:1')

bank_tract.eval('avg_approved_loan_size = sum_approved_loans/num_approved_loans',inplace=True)

bank_tract.to_csv('../input_data_clean/bank_tract.csv',index=False)


## Merge the bank-tract data with the census tract level info

In [ ]:
if not os.path.exists('../input_data_clean/bank_tract_clean_WITH_CENSUS.csv'):
        
    (
        pd.read_csv('../input_data_clean/census_clean.csv')
        .merge(pd.read_csv('../input_data_clean/bank_tract_clean.csv'),
               on = "census_tract", 
               how = "inner",
               validate = "1:m")
        .to_csv('../input_data_clean/bank_tract_clean_WITH_CENSUS.csv')
    )